# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [30]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [14]:
list(c.find({"name":"Babelgum"}, projection = {"_id":0, "name":1}))[:5]

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [29]:
list(c.find({"number_of_employees":{"$gt": 5000}}, projection = {"_id":0, "name":1, "number_of_employees":1}).sort("number_of_employees", -1))[:20]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [99]:
list(c.find({"founded_year":{"$gte": 2000, "$lte":2005}}, projection = {"_id":0, "name":1, "founded_year":1}))[:5]

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [98]:
condition_1 = {"ipo.valuation_amount": {"$gt": 100000000}}
condition_2 = {"founded_year": {"$lt": 2010}}
projection = {"_id":0, "name":1, "ipo":1}

list(c.find({"$and": [condition_1, condition_2]}, projection = projection).sort("ipo.value_amount", -1))[:5]

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000.0,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [36]:
condition_1 = {"number_of_employees": {"$lt": 1000}}
condition_2 = {"founded_year": {"$lt": 2010}}
projection = {"_id":0, "name":1, "ipo":1}

list(c.find({"$and": [condition_1, condition_2]}, projection = projection).sort("number_of_employees", -1))[:10]

[{'name': 'Infinera Corporation',
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'USD',
   'pub_year': 2007,
   'pub_month': 6,
   'pub_day': 7,
   'stock_symbol': 'INFN'}},
 {'name': 'Box', 'ipo': None},
 {'name': 'NorthPoint Communications Group', 'ipo': None},
 {'name': '888 Holdings', 'ipo': None},
 {'name': 'Forrester Research',
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'JPY',
   'pub_year': 1987,
   'pub_month': 6,
   'pub_day': 16,
   'stock_symbol': 'FORR'}},
 {'name': 'Workday',
  'ipo': {'valuation_amount': None,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 10,
   'pub_day': 15,
   'stock_symbol': 'NYSE:WDAY'}},
 {'name': 'InMobi', 'ipo': None},
 {'name': 'Relax Solutions Pvt Ltd', 'ipo': None},
 {'name': 'SonicWALL', 'ipo': None},
 {'name': 'Webmetrics', 'ipo': None}]

### 6. All the companies that don't include the `partners` field.

In [37]:

list(c.find({"partners":{"$exists": "False"}}, projection = {"_id":0, "name":1}))[:5]

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [42]:
list(c.find({"category_code":{"$type": "null"}}, projection = {"_id":0, "name":1}))[:5]

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [44]:

list(c.find({"number_of_employees": {"$gte": 2000, "$lte":2005}}, projection = {"_id":0, "name":1, "number_of_employees":1}))[:5]

[{'name': 'Netflix', 'number_of_employees': 2000},
 {'name': 'Razorfish', 'number_of_employees': 2000},
 {'name': 'Aspect Software', 'number_of_employees': 2000},
 {'name': 'WJT Global Solutions', 'number_of_employees': 2000},
 {'name': 'CEGID', 'number_of_employees': 2000}]

### 9. Order all the companies by their IPO price in a descending order.

In [53]:

list(c.find(projection = {"_id": 0, "name": 1, "ipo.valuation_amount":1}).sort("ipo.valuation_amount", -1))[:5]

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000.0}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000.0}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000.0}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000.0}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000.0}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [54]:
list(c.find(projection = {"_id": 0, "name": 1, "number_of_employees":1}).sort("number_of_employees", -1))[:5]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [91]:
list(c.find({"founded_month":{"$gte":6}}, projection = {"_id": 0, "name": 1, "founded_month":1}))[:1000]

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6},
 {'name': 'Fox Interactive Media', 'founded_month': 6},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Technorati', 'founded_month': 6},
 {'name': 'Sparter', 'founded_month': 6},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'Google', 'founded_month': 9},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'del

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [97]:
condition_2 = {"founded_year": {"$lt": 2000}}
condition_1 = {"acquisition.price_amount": {"$gt": 10000000}}
projection = {"_id":0, "name":1, "acquisition.price_amount":1}

list(c.find({"$and": [condition_1, condition_2]}, projection = projection).sort("acquisition.price_amount", -1))[:5]

[{'name': 'BEA Systems', 'acquisition': {'price_amount': 8500000000.0}},
 {'name': 'Navteq', 'acquisition': {'price_amount': 8100000000.0}},
 {'name': 'Sun Microsystems', 'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'Pixar', 'acquisition': {'price_amount': 7400000000.0}},
 {'name': 'LSI', 'acquisition': {'price_amount': 6600000000.0}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [96]:
projection = {"_id":0, "name":1, "acquisition":1}

list(c.find({"acquisition.acquired_year": {"$gt": 2010}}, projection = projection).sort("acquisition.price_amount", -1))[:5]

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000.0,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name':

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [76]:
list(c.find( projection = {"_id": 0, "name": 1, "founded_year":1}).sort("founded_year", -1))[:10]

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [79]:
list(c.find({"founded_day":{"$lte":7}}, projection = {"_id": 0, "name": 1, "founded_day":1}).sort("acquisition.price_amount", -1))[:10]

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'BlueLithium', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [82]:
list(c.find({"$and": [{"category_code":'web'},{"number_of_employees":{"$gt":4000}}]}, projection = {"_id":0, "name":1}).sort("number_of_employees", 1))[:5]

[{'name': 'Expedia'},
 {'name': 'AOL'},
 {'name': 'Webkinz'},
 {'name': 'Rakuten'},
 {'name': 'Los Angeles Times Media Group'}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [86]:
condition_2 = {"acquisition.price_currency_code": "EUR"}
condition_1 = {"acquisition.price_amount": {"$gt": 10000000}}
projection = {"_id":0, "name":1}

list(c.find({"$and": [condition_1, condition_2]}, projection = projection))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'},
 {'name': 'Tuenti Technologies'},
 {'name': 'BioMed Central'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [90]:
list(c.find({"acquisition.acquired_month":{"$lte":3}}, projection = {"_id": 0, "name": 1, "acquisition":1}))[:10]

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [100]:
condition_1 = {"founded_year": {"$gte": 2000, "$lte": 2010}}
condition_2 = {"acquisition.acquired_year": {"$gte": 2011}}
projection = {"_id":0, "name":1}

list(c.find({"$and": [condition_1, condition_2]}, projection = projection))[:5]

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'Kyte'},
 {'name': 'Jingle Networks'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [101]:

list(c.find({"deadpooled_year": { "$ne": None }}, projection = {"_id":0, "name":1, "deadpooled_year":1}))[:5]

[{'name': 'Wetpaint', 'deadpooled_year': 1},
 {'name': 'AdventNet', 'deadpooled_year': 2},
 {'name': 'Zoho', 'deadpooled_year': 3},
 {'name': 'Omnidrive', 'deadpooled_year': 2008},
 {'name': 'Babelgum', 'deadpooled_year': 2013}]

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```